<a href="https://colab.research.google.com/github/xszbencex/SZFM_2021_10_GitGud_Big/blob/main/SZFM_2021_10_GitGud_Big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
### **COVID19 prediction and forecasting using machine learning**
---

> ### *Import all needed packages/libraries*

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import datetime as dt

from datetime import timedelta 
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR 
from statsmodels.tsa.api import Holt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
url = 'https://raw.githubusercontent.com/xszbencex/SZFM_2021_10_GitGud_Big/main/Dataset/covid_19_data.csv' #github-on van eltárolva az adathalmazunk
covid = pd.read_csv(url)

In [ ]:
covid.tail(20) #utolso 20 sor kiirasa

In [ ]:
covid.head(20) #elso 20 sor kiirasa

In [ ]:
print("Size/Shape of the dataset",covid.shape) #shape()- visszaadja hány sort/oszlopot tart. az adatbázisunk
print("Checking for null values",covid.isnull().sum()) #isnull().sum() - megadja oszloponkent hany 0 ertek talalhato az adatbazisban
print("Checking Data-type",covid.dtypes) #dtypes() - oszloponként adatok típusa

In [ ]:
covid.drop(["SNo"],1,inplace=True) #kitoroljuk sz SNo oszlopot

In [ ]:
covid.isnull().sum()

In [ ]:
covid["ObservationDate"] = pd.to_datetime(covid["ObservationDate"]) #object tipust atalakitja datumma